In [2]:
. ./cmd.sh
[ -f path.sh ] && . ./path.sh
set -e

In [1]:
head -n 30 steps/decode_sgmm2.sh

#!/bin/bash

# Copyright 2012  Johns Hopkins University (Author: Daniel Povey).  Apache 2.0.

# This script does decoding with an SGMM system, with speaker vectors.
# If the SGMM system was
# built on top of fMLLR transforms from a conventional system, you should
# provide the --transform-dir option.

# Begin configuration section.
stage=1
transform_dir=    # dir to find fMLLR transforms.
nj=4 # number of decoding jobs.
acwt=0.1  # Just a default value, used for adaptation and beam-pruning..
cmd=run.pl
beam=13.0
gselect=15  # Number of Gaussian-selection indices for SGMMs.  [Note:
            # the first_pass_gselect variable is used for the 1st pass of
            # decoding and can be tighter.
first_pass_gselect=3 # Use a smaller number of Gaussian-selection indices in
            # the 1st pass of decoding (lattice generation).
max_active=7000
max_mem=50000000
#WARNING: This option is renamed lattice_beam (it was renamed to follow the naming
#         in the other scripts
lattice_be

In [2]:
head -n 30 steps/decode_fmllr.sh

#!/bin/bash

# Copyright 2012-2015  Johns Hopkins University (Author: Daniel Povey)

# Decoding script that does fMLLR.  This can be on top of delta+delta-delta, or
# LDA+MLLT features.

# There are 3 models involved potentially in this script,
# and for a standard, speaker-independent system they will all be the same.
# The "alignment model" is for the 1st-pass decoding and to get the
# Gaussian-level alignments for the "adaptation model" the first time we
# do fMLLR.  The "adaptation model" is used to estimate fMLLR transforms
# and to generate state-level lattices.  The lattices are then rescored
# with the "final model".
#
# The following table explains where we get these 3 models from.
# Note: $srcdir is one level up from the decoding directory.
#
#   Model              Default source:
#
#  "alignment model"   $srcdir/final.alimdl              --alignment-model <model>
#                     (or $srcdir/final.mdl if alimdl absent)
#  "adaptation model"  $srcdir/final.mdl           

In [3]:
mkdir -p exp/asr_aligned

In [13]:
. ./cmd.sh
[ -f path.sh ] && . ./path.sh
set -e

./steps/decode_fmllr.sh --nj 8 --cmd run.pl \
 exp/tri3/graph data/asr_dir exp/tri3/asr_aligned

./steps/decode_fmllr.sh --nj 8 --cmd run.pl exp/tri3/graph data/asr_dir exp/tri3/asr_aligned
steps/decode.sh --scoring-opts  --num-threads 1 --skip-scoring false --acwt 0.083333 --nj 8 --cmd run.pl --beam 10.0 --model exp/tri3/final.alimdl --max-active 2000 exp/tri3/graph data/asr_dir exp/tri3/asr_aligned.si
decode.sh: feature type is lda
steps/diagnostic/analyze_lats.sh --cmd run.pl exp/tri3/graph exp/tri3/asr_aligned.si
analyze_phone_length_stats.py: WARNING: optional-silence sil is seen only 0.0% of the time at utterance begin.  This may not be optimal.
analyze_phone_length_stats.py: WARNING: optional-silence sil is seen only 0.0% of the time at utterance end.  This may not be optimal.
steps/diagnostic/analyze_lats.sh: see stats in exp/tri3/asr_aligned.si/log/analyze_alignments.log
Overall, lattice depth (10,50,90-percentile)=(9,39,164) and mean=75.5
steps/diagnostic/analyze_lats.sh: see stats in exp/tri3/asr_aligned.si/log/analyze_lattice_depth_stats.log
score.sh: no such file data

In [14]:
. ./cmd.sh
[ -f path.sh ] && . ./path.sh
set -e

steps/decode_sgmm2.sh --nj 8 --cmd run.pl \
 exp/sgmm2_4/graph data/asr_dir \
 exp/sgmm2_4/asr_aligned

steps/decode_sgmm2.sh --nj 8 --cmd run.pl exp/sgmm2_4/graph data/asr_dir exp/sgmm2_4/asr_aligned
steps/decode_sgmm2.sh: feature type is lda
# sgmm2-acc-stats --utt2spk=ark:data/train/split8/1/utt2spk --update-flags=vwcSt "--gselect=ark,s,cs:gunzip -c exp/sgmm2_4/gselect.1.gz|" --rand-prune=0.1 exp/sgmm2_4/0.mdl "ark,s,cs:apply-cmvn  --utt2spk=ark:data/train/split8/1/utt2spk scp:data/train/split8/1/cmvn.scp scp:data/train/split8/1/feats.scp ark:- | splice-feats --left-context=3 --right-context=3 ark:- ark:- | transform-feats exp/tri3_ali/final.mat ark:- ark:- | transform-feats --utt2spk=ark:data/train/split8/1/utt2spk ark,s,cs:exp/tri3_ali/trans.1 ark:- ark:- |" "ark,s,cs:gunzip -c exp/sgmm2_4/ali.1.gz | ali-to-post ark:- ark:-|" exp/sgmm2_4/0.1.acc 
sgmm2-acc-stats --utt2spk=ark:data/train/split8/1/utt2spk --update-flags=vwcSt '--gselect=ark,s,cs:gunzip -c exp/sgmm2_4/gselect.1.gz|' --rand-prune=0.1 exp/sgmm2_4/0.mdl 'ark,s,cs:apply-cmvn  --utt2spk=ark:data/train/split8/1/utt2spk scp:d

In [40]:
steps/decode.sh --nj 8 --cmd run.pl \
 exp/tri2/graph data/asr_dir exp/tri2/asr_aligned

steps/decode.sh --nj 8 --cmd run.pl exp/tri2/graph data/asr_dir exp/tri2/asr_aligned
decode.sh: feature type is lda
steps/diagnostic/analyze_lats.sh --cmd run.pl exp/tri2/graph exp/tri2/asr_aligned
steps/diagnostic/analyze_lats.sh: see stats in exp/tri2/asr_aligned/log/analyze_alignments.log
Overall, lattice depth (10,50,90-percentile)=(9,40,154) and mean=70.6
steps/diagnostic/analyze_lats.sh: see stats in exp/tri2/asr_aligned/log/analyze_lattice_depth_stats.log
score.sh: no such file data/asr_dir/text
steps/decode.sh: Scoring failed. (ignore by '--skip-scoring true')


: 1

In [15]:
steps/get_ctm.sh data/asr_dir data/lang exp/sgmm2_4/asr_aligned

steps/get_ctm.sh data/asr_dir data/lang exp/sgmm2_4/asr_aligned


In [16]:
steps/get_prons.sh data/asr_dir data/lang exp/sgmm2_4/asr_aligned

steps/get_prons.sh data/asr_dir data/lang exp/sgmm2_4/asr_aligned
steps/get_prons.sh: exp/sgmm2_4/asr_aligned/lat.1.gz exists, so starting from lattices.
steps/get_prons.sh: done writing prons to exp/sgmm2_4/asr_aligned/prons.*.gz, silence counts in 
steps/get_prons.sh: exp/sgmm2_4/asr_aligned/sil_counts_nowb.txt and pronunciation counts in 
steps/get_prons.sh: exp/sgmm2_4/asr_aligned/pron_counts.{int,txt}
steps/get_prons.sh: ... and also in exp/sgmm2_4/asr_aligned/pron_counts_nowb.txt


In [17]:
ls exp/sgmm2_4/asr_aligned

gselect.1.gz  lat.8.gz                     prons.8.gz  score_8
gselect.2.gz  log                          score_10    score_9
gselect.3.gz  num_jobs                     score_11    scoring
gselect.4.gz  pron_bigram_counts_nowb.txt  score_12    sil_counts_nowb.txt
gselect.5.gz  pron_counts.int              score_13    vecs.1
gselect.6.gz  pron_counts.txt              score_14    vecs.2
gselect.7.gz  pron_counts_nowb.txt         score_15    vecs.3
gselect.8.gz  pron_perutt_nowb.txt         score_16    vecs.4
lat.1.gz      prons.1.gz                   score_17    vecs.5
lat.2.gz      prons.2.gz                   score_18    vecs.6
lat.3.gz      prons.3.gz                   score_19    vecs.7
lat.4.gz      prons.4.gz                   score_20    vecs.8
lat.5.gz      prons.5.gz                   score_5
lat.6.gz      prons.6.gz                   score_6
lat.7.gz      prons.7.gz                   score_7


In [21]:
head exp/sgmm2_4/asr_aligned/score_20/asr_dir.ctm
tail exp/sgmm2_4/asr_aligned/score_20/asr_dir.ctm

S02_A-Monstrous-Shadow 1 0.000 0.120 sil 
S02_A-Monstrous-Shadow 1 0.120 0.060 ax 
S02_A-Monstrous-Shadow 1 0.180 0.140 m 
S02_A-Monstrous-Shadow 1 0.320 0.060 ow 
S02_A-Monstrous-Shadow 1 0.380 0.050 n 
S02_A-Monstrous-Shadow 1 0.430 0.070 sh 
S02_A-Monstrous-Shadow 1 0.500 0.110 epi 
S02_A-Monstrous-Shadow 1 0.610 0.030 w 
S02_A-Monstrous-Shadow 1 0.640 0.100 uw 
S02_A-Monstrous-Shadow 1 0.740 0.060 n 
S28_Try-To-Recall 1 2.400 0.110 k 
S28_Try-To-Recall 1 2.510 0.240 ow 
S28_Try-To-Recall 1 2.750 0.140 n 
S28_Try-To-Recall 1 2.890 0.050 ix 
S28_Try-To-Recall 1 2.940 0.060 cl 
S28_Try-To-Recall 1 3.000 0.190 k 
S28_Try-To-Recall 1 3.190 0.030 l 
S28_Try-To-Recall 1 3.220 0.560 w 
S28_Try-To-Recall 1 3.780 0.110 el 
S28_Try-To-Recall 1 3.890 0.090 sil 


In [23]:
head -n20 exp/sgmm2_4/asr_aligned/score_20/asr_dir.ctm

S02_A-Monstrous-Shadow 1 0.000 0.120 sil 
S02_A-Monstrous-Shadow 1 0.120 0.060 ax 
S02_A-Monstrous-Shadow 1 0.180 0.140 m 
S02_A-Monstrous-Shadow 1 0.320 0.060 ow 
S02_A-Monstrous-Shadow 1 0.380 0.050 n 
S02_A-Monstrous-Shadow 1 0.430 0.070 sh 
S02_A-Monstrous-Shadow 1 0.500 0.110 epi 
S02_A-Monstrous-Shadow 1 0.610 0.030 w 
S02_A-Monstrous-Shadow 1 0.640 0.100 uw 
S02_A-Monstrous-Shadow 1 0.740 0.060 n 
S02_A-Monstrous-Shadow 1 0.800 0.220 sh 
S02_A-Monstrous-Shadow 1 1.020 0.340 el 
S02_A-Monstrous-Shadow 1 1.360 0.060 n 
S02_A-Monstrous-Shadow 1 1.420 0.120 f 
S02_A-Monstrous-Shadow 1 1.540 0.200 ow 
S02_A-Monstrous-Shadow 1 1.740 0.030 n 
S02_A-Monstrous-Shadow 1 1.770 0.040 cl 
S02_A-Monstrous-Shadow 1 1.810 0.140 k 
S02_A-Monstrous-Shadow 1 1.950 0.030 w 
S02_A-Monstrous-Shadow 1 1.980 0.180 ah 


In [9]:
python ./local/ctm2phn.py exp/tri2/asr_aligned/score_20/asr_dir.ctm exp/tri2/asr_aligned/aligned